In [127]:
import sys
dll_path = r'C:\Users\bkmz1\PycharmProjects\manipulator\dobot_lib'
sys.path.insert(1,dll_path)
import DobotDllType as dType
from math import pi
import numpy as np
import time
import scipy.optimize as optimize
from math import dist, atan2
from itertools import combinations, islice
import pickle
import ipywidgets as widgets
from typing import Optional

api = dType.load(dll_path)

The dll you are using is 64-bit. In order to run smoothly, please ensure that your python environment is also 64-bit.
The python environment is： ('64bit', 'WindowsPE')


In [128]:
def coords_from_chess_pos(pos):
    # from chess pos (e.g. 'e4') into local chess coords
    # letters - xi axis, numbers - eta. Origin at center of a1
    if isinstance(pos, str):
        char, num = pos.lower()
        xi = ord(char) - ord('a')
        eta = int(num) - 1
        return np.array([xi, eta], dtype=float)
    else:
        return np.array(list(map(coords_from_chess_pos, pos)))
    

def find_intersections(o1, r1, o2, r2, eps=.1):
    # finds intersection points of 2 circles
    # if no intersection, raises exception
    o1, o2 = numpify(o1, o2)
    d = dist(o1, o2)
    assert d <= r1+r2
    assert d > eps
    a = (r1**2-r2**2+d**2)/(2*d)
    h = (r1**2 - a**2)**.5
    P = (o2-o1)*a/d+o1
    perp_ = perp(o2-o1)
    I1, I2 = P+h*perp_, P-h*perp_
    return I1, I2

def normalize(vec):
    return np.array(vec)/length(vec)

def perp(vec):
    return normalize(vec)[::-1]*np.array([-1,1])

def length(vec):
    return dist(vec, [0]*len(vec))

def numpify(*args):
    arrays = [0]*len(args)
    for i, arg in enumerate(args):
        arrays[i] = arg.astype(np.float32) if isinstance(arg, np.ndarray) else np.array(arg, dtype=np.float32)
    return arrays

def find_angle(vec_a, vec_b):
    """
    finds angle from vec_b to vec_a in [0; 2pi) (counterclockwise)
    """
    return (atan2(*vec_a)-atan2(*vec_b))%(2*pi)

def find_circles(A_b, B_b, A_d, B_d):
    # find center and radius of circle, given coords in board frame and dobot frame
    # returns o1, o2, r
    A_b, B_b, A_d, B_d = numpify(A_b, B_b, A_d, B_d)
    a = length(A_d)
    b = length(B_d)
    c = length(A_d - B_d)
    cos_alpha = (a**2+b**2-c**2)/2/a/b
    sin_alpha = (1-cos_alpha**2)**.5
    AB_b = B_b-A_b
    l = length(AB_b)
    r = l/2/sin_alpha
    normal = r*cos_alpha*perp(AB_b)
    M = (A_b+B_b)/2
    o1, o2 = M+normal, M-normal
    return o1, o2, r

def count_coincides(A, points, eps = .1):
    return len([0 for point in points if dist(A, point) < eps])

def is_clockwise(vec1, vec2):
    # finds if vec2 is rotated clockwise from vec1
    cross = vec1[0]*vec2[1] - vec1[1]*vec2[0]
    assert cross != 0
    return cross < 0

def find_origin(board_coords, dobot_coords, j1_positions):
    # find center, given coords of 3 points
    # points all have different angle for 1st joint
    assert len(board_coords) == len(dobot_coords) == 3
    circles = []
    board_coords = list(map(coords_from_chess_pos, board_coords))
    for i, j in combinations([0,1,2], 2):
        o1, o2, r = find_circles(board_coords[i], board_coords[j], dobot_coords[i], dobot_coords[j])
        circles.extend(([o1, r], [o2, r]))
    inters = []
    for circ1, circ2 in combinations(circles, 2):
        inters.extend(find_intersections(*circ1, *circ2))
    centers = [p for p in inters if count_coincides(p, inters) == 3]
    c0 = centers[0]
    eps = .1
    for c in centers:
        if dist(c0, c) > eps:
            c1 = c
            break
    board_coords.append(board_coords[0])
    j1_positions = j1_positions + [j1_positions[0]]
    for center in (c0, c1):
        vectors = [b_point - center for b_point in board_coords]
        for v1, v2, j_val1, j_val2 in zip(vectors[:-1], vectors[1:], j1_positions[:-1], j1_positions[1:]):
            # clockwise rotation is changing j1 angle to lesser angle
            if not is_clockwise(v1, v2) == (j_val2 - j_val1 < 0):
                break
        else:
            return center
    raise ValueError('bad points passed')

def find_rot_angle(p_b, q_a, origin):
    """
    returns rotation matrix R_ab
    p_b - coordinates of point in board frame
    q_a - coordinates shown by dobot
    origin - coordinates of dobot's origin in board frame
    """
    p_b = coords_from_chess_pos(p_b)
    p_b, q_a, origin = numpify(p_b, q_a, origin)
    # same vector in different frames
    v_b = normalize(p_b-origin)
    v_a = normalize(q_a)
    
    angle = find_angle(v_b, v_a)
    return angle

def find_l(board_coords, dobot_coords, origin):
    """
    board_coords - coords of 3 points in board's frame
    dobot_coords - coords shown by dobot
    origin - origin of dobot in board's frame
    
    returns d - offset, l - size of tile (both in mm, if given dobot coords are in mm)
    """
    board_coords = list(map(coords_from_chess_pos, board_coords))
    dobot_coords, origin = numpify(dobot_coords, origin)
    lengths_b = [length(point-origin) for point in board_coords]
    lengths_a = [length(point) for point in dobot_coords]
    def la_from_lb(l_b, l):
        return l*l_b
    popt, _ = optimize.curve_fit(la_from_lb, lengths_b, lengths_a, [28])
    print('d, l optimization error:', sum([abs(la_from_lb(l_b, *popt) - l_a) for l_b, l_a in zip(lengths_b, lengths_a)]))
    return popt

def rot_matr(phi):
    c, s = np.cos(phi), np.sin(phi)
    return np.array([[c, -s],
                     [s, c]])

In [129]:
def calibrate_by_optimizing(board_xy, dobot_xy, defaults=None):
    def get_coords(x):
        phi, o_x, o_y, tile_size, offset = x
        summ = 0
        for p_b, p_a in zip(board_xy, dobot_xy):
            c, s = np.cos(phi), np.sin(phi)
            R_ab = np.array([[c, -s],
                             [s, c]])
            origin = np.array([o_x, o_y])
            t = R_ab@(coords_from_chess_pos(p_b) - origin)
            p_a_calc = t*tile_size - offset * normalize(t)
            summ += dist(p_a, p_a_calc)
        return summ
    if defaults is None:
        defaults = [3.14/2,4, 16, 28, -30]
    optimizer = optimize.minimize(get_coords, defaults,  method='Nelder-Mead', options={'maxiter':10**10})
    phi, o_x, o_y, tile_size, offset = optimizer.x
    R_ab = rot_matr(phi)
    print(f'error distance is: {get_coords(optimizer.x)}')
    return R_ab, np.array([o_x, o_y]), tile_size, offset

def calibrate_by_optimizing_2(board_xy, dobot_xy, tile_size):
    def get_coords(x):
        phi, o_x, o_y, offset = x
        summ = 0
        for p_b, p_a in zip(board_xy, dobot_xy):
            c, s = np.cos(phi), np.sin(phi)
            R_ab = np.array([[c, -s],
                             [s, c]])
            origin = np.array([o_x, o_y])
            t = R_ab@(coords_from_chess_pos(p_b) - origin)
            p_a_calc = t*tile_size - offset * normalize(t)
            summ += dist(p_a, p_a_calc)
        return summ
    optimizer = optimize.minimize(get_coords, [3.14/2, 4, 16, 0],  method='Nelder-Mead', options={'maxiter':10**10})
    phi, o_x, o_y, offset = optimizer.x
    c, s = np.cos(phi), np.sin(phi)
    R_ab = rot_matr(phi)
    print(f'error distance is: {get_coords(optimizer.x)}, {offset=}')
    return R_ab, np.array([o_x, o_y]), offset

In [231]:
class Optimizer:
    def __init__(self):
        self.twist = None
    
    @staticmethod
    def kabsch_umeyama(board_coords, dobot_coords):
        assert dobot_coords.shape == board_coords.shape
        n, m = dobot_coords.shape

        E_dobot_coords = np.mean(dobot_coords, axis=0)
        E_board_coords = np.mean(board_coords, axis=0)
        # var_dobot_coords = np.var(dobot_coords, axis=0)
        var_dobot_coords = np.mean(np.linalg.norm(dobot_coords - E_dobot_coords, axis=1) ** 2)

        H = ((dobot_coords - E_dobot_coords).T @ (board_coords - E_board_coords)) / n
        U, D, VT = np.linalg.svd(H)
        d = np.sign(np.linalg.det(U) * np.linalg.det(VT))
        S = np.diag([1] * (m - 1) + [d])

        R = U @ S @ VT
        # scaling = np.diag(var_dobot_coords / np.trace(np.diag(D) @ S))
        # scaling = var_dobot_coords / np.trace(np.diag(D) @ S)
        scaling = 29.07
        offset = (E_dobot_coords - scaling * R @ E_board_coords)
        twist = np.vstack([
            np.hstack([scaling * R, offset.reshape(-1, 1)]),
            [0, 0, 1]
        ])
        return twist
    
    def fit(self, board_coords: np.ndarray, dobot_coords: np.ndarray):
        self.twist = self.kabsch_umeyama(board_coords, dobot_coords)
        return self
    
    def transform(self, board_coords: np.ndarray):
        only_one = board_coords.ndim == 1
        if only_one:
            board_coords = board_coords.reshape(1, -1)
        homog_board_coords = np.vstack([board_coords.T, np.ones(len(board_coords))])
        dobot_coords = (self.twist @ homog_board_coords).T[:, :2]
        if only_one:
            return dobot_coords[0]
        return dobot_coords
    
    @staticmethod
    def coords_from_chess_poses(pos):
        # from chess pos (e.g. 'e4') into local chess coords
        # letters - xi axis, numbers - eta. Origin at center of a1
        if isinstance(pos, str):
            char, num = pos.lower()
            xi = ord(char) - ord('a')
            eta = int(num) - 1
            return np.array([xi, eta], dtype=float)
        else:
            return np.array(list(map(coords_from_chess_pos, pos)))

In [272]:
class Pieces:
    def __init__(self, dobot):
        self.dobot = dobot
        
        # additional offset when carrying pieces
        self.default_offset = 20
        self.zero_height = None
        self.pieces_heights = {}
    
    def copy_from(self, other):
        self.zero_height = other.zero_height
        self.pieces_heights = other.pieces_heights
        return self
    
    def get_height(self, piece: str):
        # get piece's z coordinate by its name
        if piece is None:
            return self.zero_height + self.default_offset
        else:
            return self.pieces_heights[piece]
    
    def _dobot_height(self):
        return dType.GetPose(api)[2]
    
    def calibrate(self):
        # calibrate heights of pieces
        input('Move to zero height, then press Enter')
        self.zero_height = self.dobot.get_pos()[2]
        pieces = ['pawn', 'rook', 'knight', 'bishop', 'queen', 'king']
        for piece in pieces:
            input(f"Move to {piece}'s height, then press enter")
            self.pieces_heights[piece] = self.dobot.get_pos()[2]
            print(f'Height is {self.pieces_heights[piece]}')
        
    def trans_height(self):
        # height safe for transporting a piece
        return (max(self.pieces_heights.values()) - self.zero_height)*2 + self.default_offset + self.zero_height

In [188]:
class Board:
    def __init__(self, dobot):
        self.dobot = dobot
        self.R_ab = None
        self.origin = None
        self.tile_size = None
        self.offset = None
        self.coords_data = {'bc':[], 'dc':[]}
        self.optimizer = Optimizer()
    
    def read_coords(self, num_of_points=3, rewrite_coords=False):
        board_coords, dobot_coords = [], []
        if not rewrite_coords:
            board_coords, dobot_coords = self.coords_data['bc'], self.coords_data['dc']
        self.coords_data['bc'] = board_coords
        self.coords_data['dc'] = dobot_coords
        delta = widgets.RadioButtons(options=['0.1 mm', '1 mm', '10 mm'],)
        change_coords = [widgets.Button(description=descr) for descr in ['-X', '+X', '-Y', '+Y', '-Z', '+Z']]
        record = widgets.Button(description='record (leave field empty to stop recording)')
        inp = widgets.Text(description='coords on board (e.g. "a2")')
        def stop_recording():
            for button in change_coords + [record]:
                button.disabled = True
            print('stopped recording!')
            
        def on_click_record(b, temp={'num_of_points':num_of_points}):
            if inp.value == '':
                stop_recording()
            else:
                self.coords_data['bc'].append(inp.value)
                inp.value = ''
                self.coords_data['dc'].append(dType.GetPose(api)[:4])
                temp['num_of_points'] -= 1
                if not temp['num_of_points']:
                    stop_recording()
                    
        def change_coord_on_click(b):
            mask = {
                'X': [1, 0, 0],
                'Y': [0, 1, 0],
                'Z': [0, 0, 1]
            }[b.description[1]]
            mask = np.array(mask, dtype=float)
            mask *= -1 if b.description[0] == '-' else 1
            mask *= [.1, 1, 10][delta.get_state()['index']]
            self.dobot.move_to(*(self.dobot.get_xyz() + mask))
        
        record.on_click(on_click_record)
        for button in change_coords:
            button.on_click(change_coord_on_click)
        display(delta, *change_coords, record, inp)
    
    def optimize(self):
        self.optimizer.fit(
            np.array(Optimizer.coords_from_chess_poses(self.coords_data['bc'])),
            np.array(self.coords_data['dc'])[:, :2]
        )
        
    def coord_from_board(self, board_xy: np.ndarray):
        board_xy = np.array(board_xy, dtype=float)
        return self.optimizer.transform(board_xy)

In [248]:
class Dobot:
    def __init__(self, com, copy_from=None, from_data=None):
        self.com = com
        self.connect()
        self.pieces = Pieces(self)
        self.board = Board(self)
        if copy_from is not None:
            self.copy(copy_from)
        elif from_data is not None:
            self.copy_data(data)
    
    def connect(self):
        self.disconnect()
        print(dType.ConnectDobot(api, self.com, 115200))
        dType.SetQueuedCmdClear(api)
        dType.SetQueuedCmdStartExec(api)
        dType.SetHOMEParams(api, 200, 0, 200, 200, isQueued = 1)
        dType.SetPTPJointParams(api, 200, 200, 200, 200, 200, 200, 200, 200, isQueued = 1)
        dType.SetPTPCommonParams(api, 100, 100, isQueued = 1)
    
    def disconnect(self):
        dType.DisconnectDobot(api)
    
    def __del__(self):
        dType.DisconnectDobot(api)
    
    def reconnect(self):
        self.connect()
            
    def copy(self, other_dobot):
        try:
            self.calibrate_pos(0)
        except Exception as e:
            print('exception, while trying to calibrate')
        self.pieces = other_dobot.pieces
        
    def copy_data(self, data):
        self.pieces = data['pieces']
    
    def move_to(self, x=None, y=None, z=None, j4_rot=None, timeout=.5):
        x, y, z, j4_rot = [coord_orig if coord_orig is not None else coord_new 
                              for coord_orig, coord_new in zip((x, y, z, j4_rot), self.get_pos()[:4])]
        for _ in range(2):
            try:
                dType.SetPTPCmd(api,
                    dType.PTPMode.PTPMOVLXYZMode,
                    x, y, z, j4_rot,
                    isQueued = 1,
                    timeout = timeout
                )
            except RuntimeError as e:
                print(e)
                reconnect()
                time.sleep(.1)
            else:
                return
        raise e
    
    def move_to_tile(self, board_xy, chess_piece=None, height=None, j4_rot=None):
        dobot_xy = self.board.coord_from_board(board_xy)
        cur_pos = self.get_pos()
        if height is None:
            if chess_piece is None:
                height = cur_pos[2]
            else:
                height = self.pieces.get_height(chess_piece)
        if j4_rot is None:
            j4_rot = cur_pos[3]
        self.move_to(*dobot_xy, height, j4_rot)
    
    def get_xyz(self):
        return self.get_pos()[:3]
    
    def get_pos(self):
        return np.array(dType.GetPose(api))
    
    def move_piece(self, from_coords, to_coords, piece):
        trans_height = self.pieces.trans_height()
        self.move_to(z=trans_height)
        self.move_to_tile(from_coords, height=trans_height)
        self.move_to_tile(from_coords, height=self.pieces.get_height(piece))
        dType.SetEndEffectorSuctionCup(api, 1, 1, True)
        time.sleep(3)
        
        self.move_to_tile(from_coords, height=trans_height)
        self.move_to_tile(to_coords, height=trans_height)
        self.move_to_tile(to_coords, height=self.pieces.get_height(piece))
        dType.SetEndEffectorSuctionCup(api, 1, 0, True)
        time.sleep(1)

In [263]:
db.move_to_tile([4, 4])

In [195]:
db = Dobot('COM6')

[0, 2, 0, 'DobotSerial', '0.0.0', 25, 0, 0.0]


In [201]:
# with open('coords.txt', 'w') as f:
#     print(db.board.coords_data, file=f)
#     print(db.board.coords_data)

{'bc': ['h1', 'a1', 'd5'], 'dc': [[168.06251525878906, -108.178955078125, -63.15735626220703, -33.96176528930664], [167.61117553710938, 89.9608154296875, -63.85260009765625, 26.3892765045166], [285.451416015625, 7.739808559417725, -63.33761215209961, 0.045200467109680176]]}


In [ ]:
dType.ClearAllAlarmsState(api)

In [278]:
db.reconnect()

[0, 2, 0, 'DobotSerial', '0.0.0', 31, 0, 0.0]


In [196]:
db.board.read_coords(5)

RadioButtons(options=('0.1 mm', '1 mm', '10 mm'), value='0.1 mm')

Button(description='-X', style=ButtonStyle())

Button(description='+X', style=ButtonStyle())

Button(description='-Y', style=ButtonStyle())

Button(description='+Y', style=ButtonStyle())

Button(description='-Z', style=ButtonStyle())

Button(description='+Z', style=ButtonStyle())

Button(description='record (leave field empty to stop recording)', style=ButtonStyle())

Text(value='', description='coords on board (e.g. "a2")')

In [266]:
t = db.pieces

In [276]:
db.pieces = Pieces(db).copy_from(t)

In [282]:
db.move_piece([4, 4], [7, 0], 'rook')

In [202]:
db.board.optimize()

[[7. 0.]
 [0. 0.]
 [3. 4.]] [[ 168.06251526 -108.17895508]
 [ 167.61117554   89.96081543]
 [ 285.45141602    7.73980856]]


In [14]:
dist(db.temp['dc'][0], db.temp['dc'][-1])/6

28.015946013219935

In [91]:
db.temp

{'bc': ['e8', 'g7', 'd4', 'e2'],
 'dc': [[267.1821594238281, -24.27154541015625],
  [210.69607543945312, 5.0494513511657715],
  [296.0400695800781, 87.86585235595703],
  [269.6284484863281, 145.03778076171875]],
 'j1': [-5.190655708312988,
  1.3728630542755127,
  16.53109359741211,
  28.276569366455078]}

In [23]:
dType.GetEndEffectorParams(api)

[59.70000076293945, 0.0, 0.0]

In [43]:
for i in range(3):
    vec = db.dobot_xy_from_board_xy(db.temp['bc'][i])
    print(vec)
    print(vec*(length(db.temp['dc'][i])/length(vec)))
    print(length(db.temp['dc'][i]) - length(vec))
    print()

[293.00394265 -41.00261997]
[292.9919131  -41.00093657]
-0.012146768922946194

[191.0431116  77.1240676]
[189.79341002  76.61956331]
-1.3476938230176643

[277.61739522 126.08375106]
[278.45726985 126.46519165]
0.9224349886359846



In [15]:
db.calibrate_pos(method=0)

[[-0.99731327 -0.0732546 ]
 [ 0.0732546  -0.99731327]]
[13.70218526  6.29568487]
28.22
0


In [20]:
data['temp'] = {'bc': ['c4', 'h3', 'h7', 'a6', 'b5'],
 'dc': [[259.0032653808594, -46.066925048828125],
  [284.808349609375, 94.43470001220703],
  [177.1175994873047, 97.26470184326172],
  [204.3495330810547, -105.91143798828125],
  [231.33233642578125, -75.40081787109375]],
 'j1': [-10.085293769836426,
  18.344118118286133,
  28.773529052734375,
  -27.397058486938477,
  -18.052940368652344]}

In [42]:
db.move_piece('e2', 'e4', 'pawn')
input()
db.move_piece('e7', 'e5', 'pawn')
input()
db.move_piece('e1', 'e2', 'king')
input()
db.move_piece('e8', 'e7', 'king')

pawn
pawn

pawn
pawn

king
king

king
king


In [38]:
db.move_piece('e8', 'e7', 'king')

king
king


In [47]:
db.temp

{'bc': ['e8', 'g7', 'd4', 'h1'],
 'dc': [[266.4316101074219, -17.031978607177734],
  [212.35418701171875, 9.959300994873047],
  [295.51904296875, 88.67515563964844],
  [187.85848999023438, 170.5806427001953]],
 'j1': [-3.6577281951904297,
  2.6851744651794434,
  16.702701568603516,
  42.24030685424805]}

In [48]:
with open('coords.pickle', 'wb') as f:
    pickle.dump({'temp':db.temp, 'zero_height':db.zero_height, 'heights':db.pieces_heights}, f)

In [46]:
positions = []
while input() != 'q':
    positions.append(dType.GetPose(api))
with open('positions_to_check', 'wb') as f:
    pickle.dump({'coords':positions, 'offset': dType.GetEndEffectorParams(api)}, f)










q
